In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 8,4

In [2]:
# getting the data
# Internal Data download

conv0 = pd.read_excel('C:\\Users\\z0047cfr\\Project Work\\Factory Demand Model\\Input Data\\Internal Data\\Welders converted by month for Sacramento.xlsx', sheet_name = 'Sheet1', usecols= range(0, 13))
#conv0['Year'] = conv0.Year.str.replace('FY','20')
conv0 = conv0.apply(pd.to_numeric)
#conv0.head()
conv0.columns = ['Year', 10,11,12,1,2,3,4,5,6,7,8,9]
conv0

,Year,10,11,12,1,2,3,4,5,6,7,8,9
0,2014,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0
1,2015,0,1,4,0,0,0,0,0,0,3.0,0.0,2.0
2,2016,3,1,4,0,0,0,0,0,0,3.0,0.0,2.0
3,2017,14,4,0,3,6,3,7,1,0,0.0,0.0,0.0
4,2018,3,0,0,0,0,3,0,2,2,1.0,0.0,8.0
5,2019,2,0,0,0,0,1,0,22,0,0.0,2.0,3.0
6,2020,0,1,0,21,15,6,1,0,0,NaN,NaN,NaN


The conversion data has been provided based on Fiscal year and not the calendar year, when we convert month and year into dates, we need to be careful to mark oct, nov and dec of a fiscal year (eg 2020) as the calendar year (2019).

In [3]:
conv0 = pd.melt(conv0, id_vars=['Year'], 
               value_vars=[ 10,11,12,1,2,3,4,5,6,7,8,9],
               var_name='Month', value_name='Labor_conv_count_M')
conv0.head()

,Year,Month,Labor_conv_count_M
0,2014,10,0.0
1,2015,10,0.0
2,2016,10,3.0
3,2017,10,14.0
4,2018,10,3.0


In [4]:
#conv0['Month'] = pd.to_datetime(conv0.Month, format='%M').dt.month
#conv0.head(6)

The above presented data is showing wrong Fiscal years for oct nov and dec( showing FY 15 for CY 14)  and needs to be corrected

In [5]:
def fn_name_def(fn_name):
 if fn_name['Month'] in (10, 11, 12):
   return -1
 return 0  

conv0['Addition']=conv0.apply(lambda fn_name: fn_name_def(fn_name),axis=1) 
conv0.tail()
conv0['New_Year'] = conv0.Year + conv0.Addition
conv0.head(20)

,Year,Month,Labor_conv_count_M,Addition,New_Year
0,2014,10,0.0,-1,2013
1,2015,10,0.0,-1,2014
2,2016,10,3.0,-1,2015
3,2017,10,14.0,-1,2016
4,2018,10,3.0,-1,2017
5,2019,10,2.0,-1,2018
6,2020,10,0.0,-1,2019
7,2014,11,0.0,-1,2013
8,2015,11,1.0,-1,2014
9,2016,11,1.0,-1,2015


In [6]:
# converting date to date time format 
conv0['DATE'] = pd.to_datetime((conv0.New_Year*10000+conv0.Month*100+1).apply(str),format='%Y%m%d')
conv0 = conv0.dropna(how='any', axis='rows')
conv0.dtypes
conv0.head(6)

,Year,Month,Labor_conv_count_M,Addition,New_Year,DATE
0,2014,10,0.0,-1,2013,2013-10-01
1,2015,10,0.0,-1,2014,2014-10-01
2,2016,10,3.0,-1,2015,2015-10-01
3,2017,10,14.0,-1,2016,2016-10-01
4,2018,10,3.0,-1,2017,2017-10-01
5,2019,10,2.0,-1,2018,2018-10-01


In [7]:
conv0['DATE'] = pd.to_datetime(conv0.DATE)
conv0 = conv0.set_index('DATE')
conv0 = conv0.sort_index()
conv0.tail(20)

,Year,Month,Labor_conv_count_M,Addition,New_Year
DATE,,,,,
2018-11-01,2019,11,0.0,-1,2018
2018-12-01,2019,12,0.0,-1,2018
2019-01-01,2019,1,0.0,0,2019
2019-02-01,2019,2,0.0,0,2019
2019-03-01,2019,3,1.0,0,2019
2019-04-01,2019,4,0.0,0,2019
2019-05-01,2019,5,22.0,0,2019
2019-06-01,2019,6,0.0,0,2019
2019-07-01,2019,7,0.0,0,2019


In [8]:
# select modelling period
#data = data['2014-10-01': '2020-03-01']
#data.head()
conv = conv0.drop( columns = ['Year', 'Month', 'Addition', 'New_Year'])
conv.head()

,Labor_conv_count_M
DATE,
2013-10-01,0.0
2013-11-01,0.0
2013-12-01,0.0
2014-01-01,0.0
2014-02-01,0.0


In [9]:
conv.to_pickle('conv.pkl')  # save a python data frame 
conv.to_csv('conv.csv') # Use Tab to seperate data